In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.io import wavfile
import scipy
import math
import wave
import pyaudio
import binascii

In [4]:
g = np.array([[1,1,1,0,0,0,0],[1,0,0,1,1,0,0],[0,1,0,1,0,1,0],[1,1,0,1,0,0,1]])
h = np.array([[1,0,1,0,1,0,1],[0,1,1,0,0,1,1],[0,0,0,1,1,1,1],])

def encode(orig):
    m = np.frombuffer(orig.encode(),'u1') - ord('0')
    l = len(m)
    reshape = np.reshape(m, (int(l/4), 4))
    res = np.empty(7)
    output = ''
    for r in reshape:
        bstring = np.array2string(enc(r), separator='', precision=int)[1:-1]
        output += bstring

    return output

def decode(orig):
    code = np.frombuffer(orig.encode(),'u1') - ord('0')
    res = np.empty(4)
    l = len(code)
    reshape = np.reshape(code, (int(l/7), 7))
    output = ''
    for r in reshape:
        bstring = np.array2string(dec(r), separator='', precision=int)[1:-1]
        output += bstring
    return output

def enc(m):
    en = np.dot(m, g) % 2
    return en

def dec(m):
    dec = np.dot(h, m) % 2
    index = (4*dec[2] + 2*dec[1] + dec[0])
    if index != 0:
        m[index-1] = 0 if m[index-1] == 1 else 1
    return np.array([m[2], m[4], m[5], m[6]])
def text_to_bits(text, encoding='utf-8', errors='surrogatepass'):
    bits = bin(int.from_bytes(text.encode(encoding, errors), 'big'))[2:]
    return bits.zfill(8 * ((len(bits) + 7) // 8))

def text_from_bits(bits, encoding='utf-8', errors='surrogatepass'):
    n = int(bits, 2)
    return n.to_bytes((n.bit_length() + 7) // 8, 'big').decode(encoding, errors) or '\0'


def sine(frequency, length, rate):
    length = int(length * rate)
    factor = (float(frequency) * (math.pi * 2) / rate)
    return np.sin(np.arange(length) * factor)


def play_tone(stream, frequency, length, rate=RATE):
    chunks = [sine(frequency, length, rate)]
    chunk = np.concatenate(chunks) * 0.25
    fade = 70
    fade_in = np.arange(0., 1., 1/fade)
    fade_out = np.arange(1., 0., -1/fade)
    chunk[:fade] = np.multiply(chunk[:fade], fade_in)
    chunk[-fade:] = np.multiply(chunk[-fade:], fade_out)
    return chunk


In [143]:
FORMAT = pyaudio.paInt32
CHANNELS = 1
RATE = 44100
# BASE_FREQ = 18500
# FREQ_INTERVAL = 230
# NUM_TONES = 10
# CHUNK_LEN = 0.1
# START_FREQ = 18000
# END_FREQ = 17500
class Parm:
    def __init__(self):
        self.base_freq = 17500
        self.num_tones = 10
        self.chunk_len = 0.05
        self.start_freq = 17250
        self.end_freq = 17000
        self.freq_interval = (20000 - self.base_freq)/self.num_tones
        self.ecc = False
        self.tones_per_chunk = int(self.chunk_len * RATE)

In [144]:
s = 'Oh, it\'s such a perfect day. I\'m glad I spent it with you'
dic = {}

ns =[4]
cls = [0.04]
ds = [1]


for n in ns:
    for c in cls:
        for d in ds:
            parm = Parm()
            parm.num_tones = n
            parm.chunk_len = c
            parm.tones_per_chunk = int(parm.chunk_len * RATE)
            parm.freq_interval = (20000 - parm.base_freq)/parm.num_tones
            D = d
            name = f'tests/test_n={parm.num_tones}_len={parm.chunk_len}_d={D}_ecc={parm.ecc}.wav'
            out = send(name, s, parm)
            dic[name] = out, parm


In [66]:
#transmit sound
def send(filename, st, parm):
    while len(st) % parm.num_tones != 0:
        st += ' '

    bits = text_to_bits(st)
    ret = list(bits).copy()

    if parm.ecc:
        input_vector = encode(bits)
        input_arr = list(input_vector)
    else:
        input_arr = list(bits)
    # print(binascii.crc32(st.encode('utf8')))
    p = pyaudio.PyAudio()
    stream = p.open(format=FORMAT, channels=1, rate=RATE, output=1)

    # sound = np.empty(0)
    sound = play_tone(stream, parm.start_freq, parm.chunk_len)
    count = 0
    while input_arr != []:
        new_sound = np.zeros(parm.tones_per_chunk)
        for i in range(0,parm.num_tones):
            char = input_arr.pop(0)
            count += 1
            if char == '1':
                frequency = parm.base_freq + i * parm.freq_interval
                new_sound += play_tone(stream, frequency, parm.chunk_len)
        sound = np.append(sound, new_sound)
    end_tone = play_tone(stream, parm.end_freq, parm.chunk_len)
    sound = np.append(sound,end_tone)

    wavfile.write(filename, int(RATE), sound)
    return ret
#     print(count)


In [142]:
def decode_sound(filename, th1, th2, mult, expected, parm):
    a = wavfile.read(filename)
    inp = np.array(a[1],dtype=float)

    test_freqs = [int(parm.chunk_len * (parm.base_freq + i * parm.freq_interval)) for i in range(0,parm.num_tones)]
    N=parm.tones_per_chunk
    result_vector = ''
    started = False
    
    j = 0
    while started is False:
        curr_section = inp[j:j+N]
        Y_k = np.fft.fft(curr_section)[0:int(N/2)]/N # FFT function from numpy
        Y_k[1:] = 2*Y_k[1:] # need to take the single-sided spectrum only
        Pxx = np.abs(Y_k) # be sure to get rid of imaginary part
        f = RATE*np.arange((N/2))/N; # frequency vector
        tf = int(parm.start_freq * parm.chunk_len)
        thresh = th1 * np. mean(Pxx)
        if Pxx[tf] > thresh and Pxx[tf] > (mult * Pxx[int(tf + tf/12)]) and Pxx[tf] > (mult * Pxx[int(tf - tf/12)]):
#             thresh = Pxx[tf] * 0.5
            while Pxx[tf] > thresh and Pxx[tf] > (mult * Pxx[int(tf + tf/12)]) and Pxx[tf] > (mult * Pxx[int(tf - tf/12)]):     
                curr_section = inp[j:j+N]
                Y_k = np.fft.fft(curr_section)[0:int(N/2)]/N # FFT function from numpy
                Y_k[1:] = 2*Y_k[1:] # need to take the single-sided spectrum only
                Pxx = np.abs(Y_k) # be sure to get rid of imaginary part
                f = RATE*np.arange((N/2))/N; # frequency vector
                j += 16
            started = True
        j += 16

    inp = np.delete(inp,range(0,j))
#     print(len(inp))
    for i in range(0,int(len(inp)/N)):
        curr_section = inp[N*i:N*(i+1)]

        Y_k = np.fft.fft(curr_section)[0:int(N/2)]/N # FFT function from numpy
        Y_k[1:] = 2*Y_k[1:] # need to take the single-sided spectrum only
        Pxx = np.abs(Y_k) # be sure to get rid of imaginary part
        f = RATE*np.arange((N/2))/N; # frequency vector
        tf = int(parm.end_freq * parm.chunk_len)
    #     print(np.mean(Pxx))
        thresh = th2 * np.mean(Pxx)
        if Pxx[tf] > thresh and Pxx[tf] > (mult * Pxx[int(tf + tf/12)]) and Pxx[tf] > (mult * Pxx[int(tf - tf/12)]):
    #         print(f'Finished at {N*i}')
            break
        for tf in test_freqs:
            if Pxx[tf] > thresh and Pxx[tf] > (mult * Pxx[int(tf + tf/12)]) and Pxx[tf] > (mult * Pxx[int(tf - tf/12)]):
                result_vector += '1'
            else:
                result_vector += '0'
    error = 0
    if parm.ecc:
#         print(result_vector)
        bits = str(decode(result_vector))
        print(len(bits), len(expected))
        error = scipy.spatial.distance.hamming(list(bits), expected)
#         result_string = text_from_bits(bits)
    else:
        bits = str(result_vector)
#         print(len(bits), len(expected), bits, expected)
        error = scipy.spatial.distance.hamming(list(bits), expected)
#         result_string = text_from_bits(bit)
    # print(binascii.crc32(result_string.encode('utf8')))
#     print(len(bits))
    return error, bits
    
# decode file

def recv(filename, expected, parm):
    errs = []
    for i in range(4,16,2):
        for j in range(8,54,4):
            for mult in [1.1, 1.25, 1.5]:
#                 print(i,j,mult)
                try:
                    errs.append(decode_sound(filename, i, j, mult, expected, parm))
                except Exception as e:
                    pass

    return min(errs)


In [150]:
wav_output_filename='test_n=4_len=0.04_d=1_ecc=False.wav'

out, parm = dic['tests/' + wav_output_filename]
record_time=6

p = pyaudio.PyAudio()
stream = p.open(rate=RATE, channels=CHANNELS, format=FORMAT, input=True, frames_per_buffer = 1024)
frames = [];
    
print("*start recording");    
for i in range(0,int(RATE/1024*record_time)):
    data = stream.read(1024)
    frames.append(data)
print("*done recording");
stream.stop_stream()
stream.close()
p.terminate()

wf = wave.open('rec/' + wav_output_filename, 'wb')
wf.setnchannels(CHANNELS)
wf.setsampwidth(p.get_sample_size(FORMAT))
wf.setframerate(RATE)
wf.writeframes(b''.join(frames))
wf.close()

print(len(out))
err, bits = recv('rec/' + wav_output_filename, out, parm)
print(round(err,4))

*start recording
*done recording
480
0.0667


In [118]:
### st = '''hello world'''

# if len(st) % NUM_TONES != 0:
#     print("padding")
#     st += ' '

# input_vector = text_to_bits(st)
# print(input_vector)
# code = encode(st)
# print(code)

# clean = decode('110011011100001100110010010111001100101100110011001111001100110111111101010100000000000111100011111100110111111100011110101010110011001111001100110100110001010100000000')
# print(clean)

# output_vector = text_from_bits(clean)
# print(output_vector)